# About: scpによるバックアップ

---

Moodle構築環境のデータ、設定ファイルなどのバックアップを作成し、保存先にscpで格納します。

## 概要

アプリケーションテンプレートで構築したMoodle環境のバックアップを作成します。バックアップファイルは保存先となるホストにscpで配置します。

### 前提条件


この Notebook を実行するには事前に以下の準備を行う必要があります。

* バックアップファイルをscpで保存することができるホストマシン
* Moodle構築環境からバックアップ先となるホストにSSH公開鍵認証でログインできること

### バックアップの対象

バックアップの対象を以下に示します。

* DBデータ
* アップロードファイル(moodledata)
* PHPファイル
* 各コンテナで実行しているサービスの設定ファイル
    - Moodleコンテナ (`httpd.conf`, `php.ini`, ...)
    - データベースコンテナ (`my.cnf`, ...)
* `docker-compose.yml`
* crontab
* コンテナイメージ

## パラメータ設定

### Ansibleのグループを指定する

バックアップ対象となるAnsibleのグループ名を指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group =

バックアップ対象のノードにアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

### バックアップの保存先を指定する

バックアップファイルの保存先を指定します。

バックアップファイルの保存先となるホスト名を指定してください。

In [ ]:
# (例)
# backup_host = 'backup.example.org'

backup_host =

バックアップの保存先となるディレクトリを指定してください。バックアップファイルは、次のセルで指定したディレクトリにバックアップ日時に対応するサブディレクトリを作成し、その下に保存されます。

In [ ]:
# (例)
# backup_path = f'/backup/moodle-simple/{target_group}'

backup_path =

### SSH公開鍵認証

バックアップの保存先にSSHでアクセスするためのパラメータを指定します。

バックアップ先のホストにログインする際のユーザ名を指定してください。

In [ ]:
# (例)
# backup_user = 'user01'

backup_user =

バックアップ先のホストにログインする際のSSHの秘密鍵のパスを指定してください。ここで指定するパスはMoodle構築環境のホスト環境のパスを指定する必要があります。

In [ ]:
# (例)
# backup_ssh_identity = '~/.ssh/id_rsa'

backup_ssh_identity =

指定されたパスに秘密鍵のファイルが存在していることをチェックします。次のセルを実行してエラーにならないことを確認してください。

In [ ]:
!ansible {target_group} -m shell -a 'test -f {backup_ssh_identity}'

SSHの公開鍵ペアファイルをまだ作成していない場合は、次のセルのコメント `#` を外し実行することで公開鍵認証のファイルを作成することができます。

In [ ]:
# !ansible {target_group} -m shell -a \
#     'test -f {backup_ssh_identity} || \
#     ssh-keygen -q -t rsa -N "" -f {backup_ssh_identity} \
#     && cat {backup_ssh_identity}.pub'

### チェック
Moodle構築環境からバックアップ先として指定されたホストにSSHでログインできることを確認します。

バックアップ先のホストにログインする前に `~/.ssh/known_hosts` を更新しておきます。

> 既に `~/.ssh/known_hosts` にバックアップ先のホストを登録してある場合は次のセルの実行をスキップしてください。

In [ ]:
!ansible {target_group} -m shell -a \
    'ssh-keyscan {backup_host} >> ~/.ssh/known_hosts'

バックアップ先のホストにSSHでログインしてコマンド `ls -la` を実行してみます。

In [ ]:
ssh_command = f'ssh -i {backup_ssh_identity} {backup_user}@{backup_host}'

!ansible {target_group} -m shell -a \
    '{ssh_command} ls -la'

## バックアップ

### メンテナンスモードへの切り替え

バックアップを作成する前にMoodleをメンテナンスモードに切り替えます。

メンテナンスモードのへの切り替えが必要ない場合は、次のセルの実行をスキップしてください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker-compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

### 保存先ディレクトリの作成

バックアップファイルを保存するディレクトリを作成します。

`backup_path`で指定されたディレクトリの下に、バックアップ日時に基づくサブディレクトリを作成してバックアップファイルを保存するディレクトリとします。

In [ ]:
import datetime

backup_dir = f'{backup_path}/{datetime.datetime.now().isoformat()}/'
print(backup_dir)

ディレクトリを作成します。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} mkdir -p {backup_dir}'

### DBデータのバックアップ

DBデータのバックアップを作成します。

DBデータのバックアップファイル名を確認します。

In [ ]:
db_backup = backup_dir + 'db.sql.gz'
print(db_backup)

DBデータのバックアップを作成します。

> DBコンテナは MySQL/MariaDB であることを前提としています。

In [ ]:
!ansible {target_group} -m shell -a "chdir=/srv/moodle \
    docker-compose exec db bash -c \
    'mysqldump --single-transaction -u\$MYSQL_USER -p\$MYSQL_PASSWORD \$MYSQL_DATABASE' \
    | gzip | {ssh_command} 'cat > {db_backup}'"

バックアップが作成されたことを確認するためにバックアップ先ディレクトリのファイル一覧を表示します。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} ls -la {backup_dir}'

### その他ファイルのバックアップ

DBデータ以外のファイルのバックアップを作成します。

この節で作成するバックアップの対象を以下に示します。

* アップロードファイル(moodledata)
* PHPファイル
* 各コンテナで実行しているサービスの設定ファイル
    - Moodleコンテナ (`httpd.conf`, `php.ini`, ...)
    - データベースコンテナ (`my.cnf`, ...)
* `docker-compose.yml`
* crontab
* コンテナログのlogrotate設定

> ホスト環境の `/etc/` にある設定ファイルをバックアップの対象としていません（上記に示した crontab, コンテナログのlogrotate設定を除く）。必要に応じて別途バックアップを行ってください。

まずcrontabの設定をバックアップするために、設定内容をファイルに保存します。

In [ ]:
!ansible {target_group} -m file -a "path=/srv/moodle/misc state=directory"
!ansible {target_group} -m shell -a "chdir=/srv/moodle \
    crontab -l > misc/crontab"

コンテナログのlogrotate設定をバックアップするために、設定ファイルを `/srv/moodle/misc/logrotate.d` にコピーします。

In [ ]:
!ansible {target_group} -m file -a \
    "path=/srv/moodle/misc/logrotate.d state=directory"
!ansible {target_group} -b -a \
    "cp -fp /etc/logrotate.d/httpd /etc/logrotate.d/mysql-server \
    /srv/moodle/misc/logrotate.d"

Moodleキャッシュなどのバックアップ対象から外すディレクトリを指定します。前節のバックアップ対象であるDBデータを格納しているディレクトリ `db/data`、データベースをリストアするためのSQLファイルを格納しているディレクトリ `db/sql`も除外対象としています。

In [ ]:
exclude_dir_list = [
    './moodle/data/moodledata/cache',
    './moodle/data/moodledata/localcache',
    './moodle/data/moodledata/sessions',
    './moodle/data/moodledata/temp',
    './moodle/data/moodledata/trashdir',
    './db/data',
    './db/sql',
]

除外ディレクトリのリストをファイルに書き込みます。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

tmpdir = !ansible {target_group} -a 'mktemp -d'
with TemporaryDirectory() as workdir:
    exclude_file = Path(workdir) / 'exclude'
    with exclude_file.open(mode='w') as f:
        for line in exclude_dir_list:
            print(line, file=f)
    !cat {exclude_file}
    !ansible {target_group} -m copy -a 'src={str(exclude_file)} dest={tmpdir[1]}/'

バックアップファイル名を確認します。保存先のディレクトリはDBデータのバックアップの保存先と同じディレクトリとします。

In [ ]:
moodle_backup = backup_dir + 'moodle.tar.gz'
print(moodle_backup)

バックアップを作成します。

In [ ]:
!ansible {target_group} -m shell -a \
    'bash -c "sudo tar czpf - -C /srv/moodle -X {tmpdir[1]}/exclude ." \
    | {ssh_command} "cat > {moodle_backup}"'

バックアップが作成されたことを確認するためにバックアップ先ディレクトリのファイル一覧を表示します。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} ls -la {backup_dir}'

除外ディレクトリのファイルを格納していた作業ディレクトリを削除します。

In [ ]:
!ansible {target_group} -a 'rm -r {tmpdir[1]}'

### コンテナイメージ

コンテナイメージのバックアップを作成します。

> レポジトリから取得したコンテナイメージを変更せずに利用している場合は、この節をスキップすることも可能です。

Moodle環境を構成しているコンテナイメージ名を取得します。

In [ ]:
out = !ansible {target_group} -a 'chdir=/srv/moodle docker-compose images'
images = [':'.join(x.split()[1:3]) for x in out[3:]]
for x in images:
    print(x)

バックアップの保存先となるファイル名を確認します。保存先のディレクトリは他のバックアップの保存先と同じディレクトリとします。

In [ ]:
img_backup = backup_dir + 'container-image.tar.gz'
print(img_backup)

バックアップを作成します。

In [ ]:
!ansible {target_group} -m shell \
    -a 'docker save {" ".join(images)} \
    | gzip | {ssh_command} "cat > {img_backup}"'

バックアップが作成されたことを確認するためにバックアップ先ディレクトリのファイル一覧を表示します。

In [ ]:
!ansible {target_group} -m shell -a \
    '{ssh_command} ls -la {backup_dir}'

### メンテナンスモードの解除

メンテナンスモードを解除し、バックアップ作業を終了します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker-compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'